# Import Libraries

In [ ]:
import re
import pandas as pd
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

# Load Data

In [3]:
train = pd.read_csv("/kaggle/input/classification-of-math-problems-by-kasut-academy/train.csv")
test = pd.read_csv("/kaggle/input/classification-of-math-problems-by-kasut-academy/test.csv")
sample_sub = pd.read_csv("/kaggle/input/classification-of-math-problems-by-kasut-academy/sample_submission.csv")

display(train.shape, test.shape)
display(sample_sub.head())

(10189, 2)

(3044, 2)

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [4]:
train.head()

,Question,label
0,A solitaire game is played as follows. Six di...,3
1,2. The school table tennis championship was he...,5
2,"Given that $x, y,$ and $z$ are real numbers th...",0
3,$25 \cdot 22$ Given three distinct points $P\l...,1
4,I am thinking of a five-digit number composed ...,5


In [5]:
test.head()

,id,Question
0,0,b'Solve 0 = -i - 91*i - 1598*i - 64220 for i.\n'
1,1,Galperin G.A.\n\nA natural number $N$ is 999.....
2,2,Example 7 Calculate $\frac{1}{2 \sqrt{1}+\sqrt...
3,3,"If $A$, $B$, and $C$ represent three distinct ..."
4,4,2. Calculate $1+12+123+1234+12345+123456+12345...


In [6]:
train["label"].value_counts()

label
0    2618
1    2439
5    1827
4    1712
2    1039
3     368
6     100
7      86
Name: count, dtype: int64

In [7]:
for ind, i in enumerate(train["Question"][80:100]):
    print("Q:", ind)
    print(i)
    print("\n")

Q: 0
b'Solve 19*v - 316 - 142 = 151 + 75 for v.\n'


Q: 1
10. (5 points) As shown in the figure, it is a toy clock. When the hour hand makes one complete revolution, the minute hand makes 9 revolutions. If the two hands overlap at the beginning, then when the two hands overlap again, the degree the hour hand has turned is $\qquad$ .




Q: 2
5) A cyclist climbs a hill at a constant speed $v$ and then descends the same road at a constant speed that is three times the previous speed. The average speed for the entire round trip is. .
(A) $\frac{3}{4} v$
(B) $\frac{4}{3} v$
(C) $\frac{3}{2} v$
(D) $2 v$
(E) depends on the length of the road


Q: 3
6. In $\triangle A B C$,
$$
\tan A 、(1+\sqrt{2}) \tan B 、 \tan C
$$

form an arithmetic sequence. Then the minimum value of $\angle B$ is


Q: 4
I3.1 Let $x \neq \pm 1$ and $x \neq-3$. If $a$ is the real root of the equation $\frac{1}{x-1}+\frac{1}{x+3}=\frac{2}{x^{2}-1}$, find the value of $a$
I3.2 If $b>1, f(b)=\frac{-a}{\log _{2} b}$ and $g(b)

- Many problems starts with random things in different forms in many examples like
    - numbers
    - Example ...
    - task ...
- Has links, random names
- Has names of question creators, authors etc
- Maybe extracted or scraped questions and are not derived by humans. Or these things are introduced randomly for competition purposes.

In [9]:
X = train["Question"]
y = train["label"]
display(X.shape, y.shape, X.head(2), y.head(2))

(10189,)

(10189,)

0    A solitaire game is played as follows.  Six di...
1    2. The school table tennis championship was he...
Name: Question, dtype: object

0    3
1    5
Name: label, dtype: int64

# Split data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9170,), (1019,), (9170,), (1019,))

# Create Training Pipeline with Logistic Regression

In [12]:
lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(class_weight="balanced", multi_class='ovr'))
              ])
lr.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(class_weight='balanced',
                                    multi_class='ovr'))])

# Make Predictions

In [13]:
train_preds_lr = lr.predict(X_train)
test_preds_lr = lr.predict(X_test)

# Evaluate

In [14]:
print("Logistic Regression Train Accuracy:", lr.score(X_train, y_train))
print("Logistic Regression Test Accuracy:", lr.score(X_test, y_test))

Logistic Regression Train Accuracy: 0.8742639040348964
Logistic Regression Test Accuracy: 0.7939156035328754
